<a href="https://colab.research.google.com/github/kailashnirola/python_notebooks/blob/main/breast_cancer_dataset_nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [ ]:
df.shape

(569, 33)

In [ ]:
df.drop(columns=['id', 'Unnamed: 32'], inplace=True)
df.shape

(569, 31)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,1:], df.iloc[:,0], test_size=0.2, random_state=0, stratify=df.iloc[:,0])

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((455, 30), (114, 30), (455,), (114,))

#Scaling features

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
type(X_train)

numpy.ndarray

#Encoding Categorical data to label data

In [ ]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [ ]:
type(y_test)

numpy.ndarray

In [ ]:
y_test

array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0,
       0, 0, 1, 0])

In [ ]:
X_train = torch.from_numpy(X_train)
X_test = torch.from_numpy(X_test)
y_train = torch.from_numpy(y_train)
y_test = torch.from_numpy(y_test)

In [ ]:
class Model:
  def __init__(self, X):
    self.weights = torch.randn(X.shape[1], 1, dtype=torch.float64, requires_grad=True)
    self.bias = torch.zeros(1, dtype = torch.float64, requires_grad=True)

  def forward(self, X):
    z = torch.matmul(X, self.weights) + self.bias
    y_pred = torch.sigmoid(z)
    return y_pred

  def loss_func(self, y_pred, y):
    eps = 1e-8
    y_pred = torch.clamp(y_pred, eps, 1 - eps)

    loss = -(y * torch.log(y_pred) + (1 - y)*torch.log(1 - y_pred)).mean()
    return loss

#Important Parameters

In [ ]:
learning_rate = 0.1
epochs = 20

#Training Pipelines

In [ ]:
model = Model(X_train)

for epoch in range(epochs):
  #forward
  y_pred = model.forward(X_train)
  #loss
  loss = model.loss_func(y_pred, y_train)
  #backward
  loss.backward()
  #update
  with torch.no_grad():
    model.weights -= learning_rate * model.weights.grad
    model.bias -= learning_rate * model.bias.grad

  model.weights.grad.zero_()
  model.bias.grad.zero_()
  print(f"Epochs: {epoch + 1}, Loss: {loss.item()}")


Epochs: 1, Loss: 1.7133396679433865
Epochs: 2, Loss: 1.6442455585973155
Epochs: 3, Loss: 1.5824467849689383
Epochs: 4, Loss: 1.5271418351603738
Epochs: 5, Loss: 1.4768634060315944
Epochs: 6, Loss: 1.4319890820677972
Epochs: 7, Loss: 1.3919741915224528
Epochs: 8, Loss: 1.3562997016026357
Epochs: 9, Loss: 1.324462940502847
Epochs: 10, Loss: 1.2959794670525586
Epochs: 11, Loss: 1.2703933228893054
Epochs: 12, Loss: 1.247289505317306
Epochs: 13, Loss: 1.2263031169027077
Epochs: 14, Loss: 1.207122847112063
Epochs: 15, Loss: 1.189489290736537
Epochs: 16, Loss: 1.1731898300305559
Epochs: 17, Loss: 1.1580518233059325
Epochs: 18, Loss: 1.1439354147757366
Epochs: 19, Loss: 1.130726833255769
Epochs: 20, Loss: 1.118332686239738


In [ ]:
model.weights

tensor([[ 0.9937],
        [ 1.3587],
        [-0.0954],
        [-0.0586],
        [ 0.2872],
        [-0.0532],
        [-0.9070],
        [-1.6030],
        [ 0.2100],
        [ 0.3841],
        [ 1.1926],
        [ 0.3781],
        [ 0.3056],
        [-0.8351],
        [-0.0089],
        [ 0.2498],
        [-0.3487],
        [-0.7587],
        [-0.7068],
        [ 1.3066],
        [ 1.2643],
        [-1.0906],
        [ 1.2892],
        [ 0.1757],
        [ 0.1473],
        [-1.3364],
        [-0.4677],
        [-0.9309],
        [ 0.6515],
        [ 0.8661]], dtype=torch.float64, requires_grad=True)

#Model evaluation

In [ ]:
with torch.no_grad():
  y_pred = model.forward(X_test)
  y_pred = (y_pred > 0.9).float()
  acc = (y_pred == y_test).float().mean()
  print(f"Accuracy: {acc*100: .2f}")

Accuracy:  60.39
